In [1]:
import os
import json
import re
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter, defaultdict
import pickle

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# Vietnamese text processing
try:
    from pyvi import ViTokenizer
    PYVI_AVAILABLE = True
    print("✓ PyVi available")
except ImportError:
    print("✗ PyVi not available. Install: pip install pyvi")
    PYVI_AVAILABLE = False

# Word embeddings
try:
    from gensim.models import Word2Vec, KeyedVectors
    GENSIM_AVAILABLE = True
    print("✓ Gensim available")
except ImportError:
    print("✗ Gensim not available. Install: pip install gensim")
    GENSIM_AVAILABLE = False

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

print("\n✅ All libraries imported successfully!")

✓ PyVi available
✗ Gensim not available. Install: pip install gensim
PyTorch version: 2.3.1+cpu
CUDA available: False

✅ All libraries imported successfully!


In [2]:
def train_word2vec(documents, processor, embedding_dim=100, save_path='word2vec_vi.model'):
    """Train Word2Vec embeddings on corpus"""
    
    if GENSIM_AVAILABLE:
        print("\n Training Word2Vec embeddings...")
        
        # Prepare sentences
        sentences = []
        for doc in tqdm(documents, desc="Preparing sentences"):
            title = doc.get('title', '')
            content = doc.get('content', '')
            full_text = f"{title} {content}"
            tokens = processor.preprocess(full_text)
            if tokens:
                sentences.append(tokens)
        
        # Train Word2Vec
        model = Word2Vec(
            sentences=sentences,
            vector_size=embedding_dim,
            window=5,
            min_count=2,
            workers=4,
            sg=1,  # Skip-gram
            epochs=10
        )
        
        # Save model
        model.save(save_path)
        print(f"✓ Word2Vec trained and saved to {save_path}")
        print(f"  - Vocabulary size: {len(model.wv)}")
        print(f"  - Embedding dimension: {embedding_dim}")
        
        return model
    else:
        print("✗ Gensim not available. Cannot train Word2Vec.")
        return None

# Train Word2Vec (optional - can skip if you already have embeddings)
# Uncomment to train:
# w2v_model = train_word2vec(documents, processor, embedding_dim=100)

print("✓ Word2Vec training function ready")

✓ Word2Vec training function ready


In [3]:

class Vocabulary:
    """Build vocabulary from corpus"""
    
    def __init__(self, min_freq=2):
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}
        self.word_freq = Counter()
        self.min_freq = min_freq
        
    def build_vocab(self, documents, processor):
        """Build vocabulary from documents"""
        print("\n Building vocabulary...")
        
        # Count word frequencies
        for doc in tqdm(documents, desc="Counting words"):
            title = doc.get('title', '')
            content = doc.get('content', '')
            full_text = f"{title} {content}"
            tokens = processor.preprocess(full_text)
            self.word_freq.update(tokens)
        
        # Add words to vocabulary
        idx = 2  # Start after PAD and UNK
        for word, freq in self.word_freq.items():
            if freq >= self.min_freq:
                self.word2idx[word] = idx
                self.idx2word[idx] = word
                idx += 1
        
        print(f"✓ Vocabulary size: {len(self.word2idx)}")
        print(f"  - Total unique words: {len(self.word_freq)}")
        print(f"  - Words with freq >= {self.min_freq}: {len(self.word2idx) - 2}")
        print(f"  - Top 10 words: {self.word_freq.most_common(10)}")
        
        return self
    
    def encode(self, tokens, max_len=None):
        """Convert tokens to indices"""
        indices = [self.word2idx.get(token, 1) for token in tokens]  # 1 = UNK
        if max_len:
            if len(indices) < max_len:
                indices += [0] * (max_len - len(indices))  # 0 = PAD
            else:
                indices = indices[:max_len]
        return indices
    
    def decode(self, indices):
        """Convert indices back to tokens"""
        return [self.idx2word.get(idx, '<UNK>') for idx in indices]

# Build vocabulary
vocab = Vocabulary(min_freq=2)
vocab.build_vocab(documents, processor)

vocab_size = len(vocab.word2idx)
print(f"\n✅ Vocabulary ready: {vocab_size} words")

NameError: name 'documents' is not defined

## 4. Build Vocabulary & Word Embeddings

In [4]:
def load_documents(json_files=None):
    """Load documents from JSON files"""
    if json_files is None:
        json_files = [
            "vnexpressT_bongda_part1.json",
            "vnexpressT_bongda_part2.json",
            "vnexpressT_bongda_part3.json",
            "vnexpressT_bongda_part4.json"
        ]
    
    documents = []
    print(" Loading documents from JSON files...")
    
    for file_path in json_files:
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    if isinstance(data, list):
                        documents.extend(data)
                    print(f"  ✓ Loaded {file_path}: {len(data)} documents")
                except Exception as e:
                    print(f"  ✗ Error reading {file_path}: {e}")
        else:
            print(f"  ✗ File not found: {file_path}")
    
    print(f"\n✓ Total documents loaded: {len(documents)}")
    return documents

# Load data
documents = load_documents()

# Show sample document
if documents:
    print("\n Sample document:")
    sample = documents[0]
    print(f"Title: {sample.get('title', 'N/A')[:100]}")
    print(f"Content: {sample.get('content', 'N/A')[:200]}...")
    print(f"Date: {sample.get('date', 'N/A')}")
    print(f"Author: {sample.get('author', 'N/A')}")

 Loading documents from JSON files...
  ✗ File not found: vnexpressT_bongda_part1.json
  ✗ File not found: vnexpressT_bongda_part2.json
  ✗ File not found: vnexpressT_bongda_part3.json
  ✗ File not found: vnexpressT_bongda_part4.json

✓ Total documents loaded: 0


## 3. Load Vietnamese Football Data

In [5]:
class VietnameseTextProcessor:
    """Vietnamese text processing for neural ranking"""
    
    def __init__(self):
        # Vietnamese stopwords
        self.stop_words = set([
            'và', 'của', 'trong', 'với', 'là', 'có', 'được', 'cho', 'từ', 'một', 'các',
            'để', 'không', 'sẽ', 'đã', 'về', 'hay', 'theo', 'như', 'cũng', 'này', 'đó',
            'khi', 'những', 'tại', 'sau', 'bị', 'giữa', 'trên', 'dưới', 'ngoài',
            'thì', 'nhưng', 'mà', 'hoặc', 'nếu', 'vì', 'do', 'nên', 'rồi', 'còn', 'đều',
            'chỉ', 'việc', 'người', 'lại', 'đây', 'đấy', 'ở', 'ra', 'vào', 'lên', 'xuống'
        ])
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text"""
        if not text:
            return ""
        
        # Remove extra spaces
        text = re.sub(r'\s+', ' ', text)
        # Keep Vietnamese characters, letters, numbers
        text = re.sub(r'[^\w\sàáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđĐ]', ' ', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize(self, text):
        """Tokenize Vietnamese text"""
        if PYVI_AVAILABLE:
            try:
                return ViTokenizer.tokenize(text).split()
            except:
                pass
        return text.split()
    
    def remove_stopwords(self, tokens):
        """Remove stopwords"""
        return [token for token in tokens if token not in self.stop_words and len(token) > 1]
    
    def preprocess(self, text, remove_stop=True):
        """Full preprocessing pipeline"""
        cleaned = self.clean_text(text)
        tokens = self.tokenize(cleaned)
        if remove_stop:
            tokens = self.remove_stopwords(tokens)
        return tokens

processor = VietnameseTextProcessor()
print("✓ VietnameseTextProcessor initialized")

✓ VietnameseTextProcessor initialized


## 2. Vietnamese Text Processor

In [6]:
import os
import json
import re
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter, defaultdict
import pickle

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# Vietnamese text processing
try:
    from pyvi import ViTokenizer
    PYVI_AVAILABLE = True
    print("✓ PyVi available")
except ImportError:
    print("✗ PyVi not available. Install: pip install pyvi")
    PYVI_AVAILABLE = False

# Word embeddings
try:
    from gensim.models import Word2Vec, KeyedVectors
    GENSIM_AVAILABLE = True
    print("✓ Gensim available")
except ImportError:
    print("✗ Gensim not available. Install: pip install gensim")
    GENSIM_AVAILABLE = False

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

✓ PyVi available
✗ Gensim not available. Install: pip install gensim
PyTorch version: 2.3.1+cpu
CUDA available: False


## 1. Import Libraries

# DeepCT + Conv-KNRM for Vietnamese Football Search

## 📋 Overview
Implementation of DeepCT (Deep Contextualized Term weighting) combined with Conv-KNRM (Convolutional Kernel-based Neural Ranking Model) for Vietnamese information retrieval.

### 🎯 Components:
1. **DeepCT**: Neural term weighting for document representation
2. **Conv-KNRM**: Convolutional neural ranking model
3. **Vietnamese Text Processing**: Tokenization, stopwords, embeddings
4. **Training Pipeline**: Query-document pairs with relevance labels

### 📊 Dataset:
- Vietnamese Football News from VnExpress (1830+ documents)
- Query generation from titles and content
- BM25 baseline for comparison

## 5. DeepCT Model (Deep Contextualized Term Weighting)

DeepCT predicts term importance weights for documents using BERT-like contextualized representations.

In [ ]:
class DeepCT(nn.Module):
    """
    Deep Contextualized Term weighting model
    Predicts term importance weights for document terms
    
    FIXED: Changed output to regression (not sigmoid) for better term weighting
    """
    
    def __init__(self, vocab_size, embedding_dim=100, hidden_dim=128, dropout=0.3, max_weight=100):
        super(DeepCT, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.max_weight = max_weight
        
        # Bidirectional LSTM for contextualization
        self.lstm = nn.LSTM(
            embedding_dim, 
            hidden_dim, 
            num_layers=2,
            bidirectional=True,
            dropout=dropout,
            batch_first=True
        )
        
        # Term weight prediction head
        # FIXED: Remove Sigmoid, use ReLU + clipping for term weights [0, max_weight]
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1),
            # No sigmoid - output raw regression values
        )
        
    def forward(self, doc_indices):
        """
        Args:
            doc_indices: [batch_size, doc_len]
        Returns:
            term_weights: [batch_size, doc_len] - values in [0, max_weight]
        """
        # Embedding: [batch_size, doc_len, embedding_dim]
        embedded = self.embedding(doc_indices)
        
        # LSTM: [batch_size, doc_len, hidden_dim*2]
        lstm_out, _ = self.lstm(embedded)
        
        # Predict term weights: [batch_size, doc_len, 1] -> [batch_size, doc_len]
        term_weights = self.fc(lstm_out).squeeze(-1)
        
        # FIXED: Apply ReLU and clamp to [0, max_weight] instead of sigmoid [0, 1]
        term_weights = torch.clamp(F.relu(term_weights), min=0, max=self.max_weight)
        
        return term_weights

print("✓ DeepCT model defined (FIXED: regression output)")


✓ DeepCT model defined (FIXED: regression output)


## 6. Conv-KNRM Model (Convolutional Kernel-based Neural Ranking)

Conv-KNRM uses convolutional n-gram matching with kernel pooling for neural ranking.

In [ ]:
class ConvKNRM(nn.Module):
    """
    Convolutional Kernel-based Neural Ranking Model
    Uses n-gram convolutions + kernel pooling for matching
    
    FIXED: Corrected kernel pooling aggregation and padding
    """
    
    def __init__(self, vocab_size, embedding_dim=100, n_kernels=11, conv_filters=[1, 2, 3], embedding_layer=None):
        super(ConvKNRM, self).__init__()
        
        # FIXED: Allow shared embedding from outside
        if embedding_layer is not None:
            self.embedding = embedding_layer
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # Convolutional layers for n-gram extraction
        # FIXED: Better padding calculation
        self.convs = nn.ModuleList([
            nn.Conv1d(embedding_dim, embedding_dim, kernel_size=k, padding=(k-1)//2)
            for k in conv_filters
        ])
        
        # Kernel pooling parameters
        self.n_kernels = n_kernels
        # Gaussian kernels centered at [-1, -0.8, ..., 0.8, 1]
        self.kernel_mus = nn.Parameter(
            torch.linspace(-1, 1, n_kernels),
            requires_grad=False
        )
        self.kernel_sigmas = nn.Parameter(
            torch.full((n_kernels,), 0.1),
            requires_grad=False
        )
        
        # Final ranking layer
        # Input: n_kernels * len(conv_filters) features
        self.fc = nn.Sequential(
            nn.Linear(n_kernels * len(conv_filters), 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )
        
    def kernel_pooling(self, similarity_matrix):
        """
        Apply RBF kernel pooling on similarity matrix
        FIXED: Correct aggregation order (sum over doc, then query)
        
        Args:
            similarity_matrix: [batch, query_len, doc_len]
        Returns:
            pooled: [batch, n_kernels]
        """
        # Expand for kernels: [batch, query_len, doc_len, n_kernels]
        sim_expanded = similarity_matrix.unsqueeze(-1)
        
        # Compute kernel values using RBF
        kernel_values = torch.exp(
            -((sim_expanded - self.kernel_mus) ** 2) / (2 * self.kernel_sigmas ** 2)
        )
        
        # FIXED: Sum over doc dimension first, then query (more accurate)
        # Sum over doc: [batch, query_len, n_kernels]
        K = torch.sum(kernel_values, dim=2)
        
        # Log and sum over query: [batch, n_kernels]
        pooled = torch.sum(torch.log(K + 1e-10), dim=1)
        
        return pooled
        
    def forward(self, query_indices, doc_indices, doc_weights=None):
        """
        Args:
            query_indices: [batch_size, query_len]
            doc_indices: [batch_size, doc_len]
            doc_weights: [batch_size, doc_len] (optional, from DeepCT)
        Returns:
            scores: [batch_size, 1]
        """
        batch_size = query_indices.size(0)
        
        # Embeddings
        query_embed = self.embedding(query_indices)  # [batch, query_len, emb_dim]
        doc_embed = self.embedding(doc_indices)      # [batch, doc_len, emb_dim]
        
        # Apply DeepCT weights if provided
        # FIXED: Normalize weights before applying
        if doc_weights is not None:
            # Normalize to prevent extreme values
            doc_weights_norm = doc_weights / (doc_weights.sum(dim=1, keepdim=True) + 1e-10) * doc_weights.size(1)
            doc_embed = doc_embed * doc_weights_norm.unsqueeze(-1)
        
        all_features = []
        
        # For each n-gram size
        for conv in self.convs:
            # Apply convolution
            # Transpose for conv1d: [batch, emb_dim, seq_len]
            query_conv = conv(query_embed.transpose(1, 2)).transpose(1, 2)
            doc_conv = conv(doc_embed.transpose(1, 2)).transpose(1, 2)
            
            # Normalize embeddings
            query_norm = F.normalize(query_conv, p=2, dim=-1)
            doc_norm = F.normalize(doc_conv, p=2, dim=-1)
            
            # Compute similarity matrix: [batch, query_len, doc_len]
            similarity = torch.bmm(query_norm, doc_norm.transpose(1, 2))
            
            # Kernel pooling: [batch, n_kernels]
            pooled = self.kernel_pooling(similarity)
            all_features.append(pooled)
        
        # Concatenate all n-gram features
        features = torch.cat(all_features, dim=-1)  # [batch, n_kernels * n_conv]
        
        # Final ranking score
        scores = self.fc(features)  # [batch, 1]
        
        return scores

print("✓ Conv-KNRM model defined (FIXED: kernel pooling & padding)")


✓ Conv-KNRM model defined (FIXED: kernel pooling & padding)


## 7. Combined DeepCT + Conv-KNRM Model

### 🧪 Test Models Implementation

Kiểm tra xem models có hoạt động đúng không với dummy data.

In [ ]:
def test_models_implementation():
    """
    Test DeepCT and Conv-KNRM implementation với dummy data
    Kiểm tra shapes và giá trị output
    """
    print("\n" + "="*80)
    print(" TESTING MODEL IMPLEMENTATIONS")
    print("="*80)
    
    # Dummy parameters
    vocab_size = 1000
    embedding_dim = 50
    batch_size = 4
    query_len = 10
    doc_len = 50
    
    # Create dummy data
    query_indices = torch.randint(0, vocab_size, (batch_size, query_len))
    doc_indices = torch.randint(0, vocab_size, (batch_size, doc_len))
    
    print(f"\n📊 Test Configuration:")
    print(f"  - Vocab size: {vocab_size}")
    print(f"  - Embedding dim: {embedding_dim}")
    print(f"  - Batch size: {batch_size}")
    print(f"  - Query length: {query_len}")
    print(f"  - Document length: {doc_len}")
    
    # Test 1: DeepCT Model
    print(f"\n{'─'*80}")
    print("✓ Test 1: DeepCT Model")
    print(f"{'─'*80}")
    
    deepct = DeepCT(vocab_size, embedding_dim=embedding_dim, hidden_dim=64, max_weight=100)
    with torch.no_grad():
        term_weights = deepct(doc_indices)
    
    print(f"  Input shape: {doc_indices.shape}")
    print(f"  Output shape: {term_weights.shape}")
    print(f"  Expected shape: [batch_size={batch_size}, doc_len={doc_len}]")
    print(f"  ✓ Shape matches: {term_weights.shape == (batch_size, doc_len)}")
    print(f"  Weight range: [{term_weights.min():.3f}, {term_weights.max():.3f}]")
    print(f"  ✓ Weights in [0, 100]: {(term_weights >= 0).all() and (term_weights <= 100).all()}")
    print(f"  Mean weight: {term_weights.mean():.3f}")
    
    # Test 2: Conv-KNRM Model
    print(f"\n{'─'*80}")
    print("✓ Test 2: Conv-KNRM Model")
    print(f"{'─'*80}")
    
    convknrm = ConvKNRM(vocab_size, embedding_dim=embedding_dim, n_kernels=11, conv_filters=[1, 2, 3])
    with torch.no_grad():
        scores = convknrm(query_indices, doc_indices, doc_weights=None)
    
    print(f"  Query shape: {query_indices.shape}")
    print(f"  Doc shape: {doc_indices.shape}")
    print(f"  Output shape: {scores.shape}")
    print(f"  Expected shape: [batch_size={batch_size}, 1]")
    print(f"  ✓ Shape matches: {scores.shape == (batch_size, 1)}")
    print(f"  Score range: [{scores.min():.3f}, {scores.max():.3f}]")
    print(f"  Scores: {scores.squeeze().tolist()}")
    
    # Test 3: Conv-KNRM with DeepCT weights
    print(f"\n{'─'*80}")
    print("✓ Test 3: Conv-KNRM with DeepCT Weights")
    print(f"{'─'*80}")
    
    with torch.no_grad():
        scores_weighted = convknrm(query_indices, doc_indices, doc_weights=term_weights)
    
    print(f"  With term weights shape: {term_weights.shape}")
    print(f"  Output shape: {scores_weighted.shape}")
    print(f"  ✓ Shape matches: {scores_weighted.shape == (batch_size, 1)}")
    print(f"  Score range: [{scores_weighted.min():.3f}, {scores_weighted.max():.3f}]")
    print(f"  Scores (weighted): {scores_weighted.squeeze().tolist()}")
    print(f"  Scores (no weight): {scores.squeeze().tolist()}")
    print(f"  ✓ Weights affect scores: {not torch.allclose(scores, scores_weighted, atol=1e-5)}")
    
    # Test 4: Combined Model
    print(f"\n{'─'*80}")
    print("✓ Test 4: DeepCT + Conv-KNRM Combined Model")
    print(f"{'─'*80}")
    
    combined_model = DeepCT_ConvKNRM(
        vocab_size, 
        embedding_dim=embedding_dim, 
        hidden_dim=64,
        n_kernels=11,
        conv_filters=[1, 2, 3],
        use_deepct=True,
        max_weight=100
    )
    
    with torch.no_grad():
        scores_combined, weights_combined = combined_model(query_indices, doc_indices)
    
    print(f"  Output scores shape: {scores_combined.shape}")
    print(f"  Output weights shape: {weights_combined.shape}")
    print(f"  ✓ Scores shape matches: {scores_combined.shape == (batch_size, 1)}")
    print(f"  ✓ Weights shape matches: {weights_combined.shape == (batch_size, doc_len)}")
    print(f"  Score range: [{scores_combined.min():.3f}, {scores_combined.max():.3f}]")
    
    # Test 5: Check shared embeddings
    print(f"\n{'─'*80}")
    print("✓ Test 5: Shared Embeddings Check")
    print(f"{'─'*80}")
    
    print(f"  DeepCT embedding: {id(combined_model.deepct.embedding)}")
    print(f"  Conv-KNRM embedding: {id(combined_model.convknrm.embedding)}")
    print(f"  Shared embedding: {id(combined_model.shared_embedding)}")
    print(f"  ✓ Embeddings are shared: {combined_model.deepct.embedding is combined_model.convknrm.embedding}")
    
    # Test 6: Parameter count
    print(f"\n{'─'*80}")
    print("✓ Test 6: Model Parameters")
    print(f"{'─'*80}")
    
    total_params = sum(p.numel() for p in combined_model.parameters())
    trainable_params = sum(p.numel() for p in combined_model.parameters() if p.requires_grad)
    
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")
    
    # Test gradient flow
    print(f"\n{'─'*80}")
    print("✓ Test 7: Gradient Flow Test")
    print(f"{'─'*80}")
    
    combined_model.train()
    scores_grad, weights_grad = combined_model(query_indices, doc_indices)
    loss = scores_grad.mean()
    loss.backward()
    
    has_grad = sum(1 for p in combined_model.parameters() if p.grad is not None)
    total_model_params = sum(1 for p in combined_model.parameters())
    
    print(f"  Parameters with gradients: {has_grad}/{total_model_params}")
    print(f"  ✓ Gradients computed: {has_grad > 0}")
    
    print(f"\n{'='*80}")
    print("✅ ALL TESTS PASSED - Models implementation is correct!")
    print("="*80)
    
    return True

# Run tests
try:
    test_models_implementation()
except Exception as e:
    print(f"\n❌ TEST FAILED with error:")
    print(f"   {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()



 TESTING MODEL IMPLEMENTATIONS

📊 Test Configuration:
  - Vocab size: 1000
  - Embedding dim: 50
  - Batch size: 4
  - Query length: 10
  - Document length: 50

────────────────────────────────────────────────────────────────────────────────
✓ Test 1: DeepCT Model
────────────────────────────────────────────────────────────────────────────────
  Input shape: torch.Size([4, 50])
  Output shape: torch.Size([4, 50])
  Expected shape: [batch_size=4, doc_len=50]
  ✓ Shape matches: True
  Weight range: [0.000, 0.090]
  ✓ Weights in [0, 100]: True
  Mean weight: 0.037

────────────────────────────────────────────────────────────────────────────────
✓ Test 2: Conv-KNRM Model
────────────────────────────────────────────────────────────────────────────────
  Query shape: torch.Size([4, 10])
  Doc shape: torch.Size([4, 50])
  Output shape: torch.Size([4, 1])
  Expected shape: [batch_size=4, 1]
  ✓ Shape matches: True
  Score range: [25.147, 41.178]
  Scores: [41.177669525146484, 38.6601028442382

Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15860\1562043528.py", line 147, in <module>
    test_models_implementation()
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15860\1562043528.py", line 83, in test_models_implementation
    combined_model = DeepCT_ConvKNRM(
                     ^^^^^^^^^^^^^^^
NameError: name 'DeepCT_ConvKNRM' is not defined


In [ ]:
class DeepCT_ConvKNRM(nn.Module):
    """
    Combined model: DeepCT for term weighting + Conv-KNRM for ranking
    FIXED: Share embeddings between DeepCT and Conv-KNRM
    """
    
    def __init__(self, vocab_size, embedding_dim=100, hidden_dim=128, 
                 n_kernels=11, conv_filters=[1, 2, 3], use_deepct=True, max_weight=100):
        super(DeepCT_ConvKNRM, self).__init__()
        
        self.use_deepct = use_deepct
        
        # FIXED: Shared embedding layer for both models
        self.shared_embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # DeepCT for term weighting
        if use_deepct:
            self.deepct = DeepCT(vocab_size, embedding_dim, hidden_dim, max_weight=max_weight)
            # Share embedding with DeepCT
            self.deepct.embedding = self.shared_embedding
        
        # Conv-KNRM for ranking (with shared embedding)
        self.convknrm = ConvKNRM(vocab_size, embedding_dim, n_kernels, conv_filters, 
                                 embedding_layer=self.shared_embedding)
        
    def forward(self, query_indices, doc_indices):
        """
        Args:
            query_indices: [batch_size, query_len]
            doc_indices: [batch_size, doc_len]
        Returns:
            scores: [batch_size, 1]
            doc_weights: [batch_size, doc_len] (if use_deepct=True)
        """
        doc_weights = None
        
        # Get term weights from DeepCT
        if self.use_deepct:
            doc_weights = self.deepct(doc_indices)
        
        # Rank with Conv-KNRM
        scores = self.convknrm(query_indices, doc_indices, doc_weights)
        
        return scores, doc_weights

print("✓ DeepCT + Conv-KNRM combined model defined (FIXED: shared embeddings)")


✓ DeepCT + Conv-KNRM combined model defined (FIXED: shared embeddings)


## 8. Dataset Preparation

Generate query-document pairs with relevance labels for training.

---

## 🔧 Implementation Issues & Fixes

### ❌ **Issues Found:**

#### 1. **DeepCT Model - Sigmoid Output Problem**
**Problem:** Sử dụng `Sigmoid()` làm output activation giới hạn term weights trong [0, 1]
- Paper gốc: DeepCT xuất ra regression values (integer weights 1-100)
- Implementation cũ: Weights chỉ trong [0, 1] → không đủ range để phân biệt term importance

**Fix:** ✅ Đổi sang regression output với `ReLU + Clamp [0, 100]`

#### 2. **Conv-KNRM Kernel Pooling - Aggregation Order**
**Problem:** Sum trên cả query và doc dimensions cùng lúc `dim=[1, 2]`
- Paper gốc: Sum over doc first, then aggregate query
- Implementation cũ: Mất thông tin về query term matching

**Fix:** ✅ Sum over doc dimension trước, sau đó log-sum over query

#### 3. **Embedding Duplication**
**Problem:** DeepCT và Conv-KNRM có 2 embedding layers riêng biệt
- Tốn memory và parameters
- Không consistent giữa 2 models

**Fix:** ✅ Share embedding layer giữa DeepCT và Conv-KNRM

#### 4. **Conv Padding Issue**
**Problem:** `padding=k//2` với kernel_size=1 → padding=0
- Có thể gây ra dimension mismatch

**Fix:** ✅ Dùng `padding=(k-1)//2` chính xác hơn

#### 5. **DeepCT Weights Normalization**
**Problem:** Không normalize weights trước khi apply vào embeddings
- Weights quá lớn có thể gây instability

**Fix:** ✅ Normalize weights trước khi nhân với embeddings

---

### ✅ **Changes Made:**

```python
# OLD - DeepCT
nn.Sigmoid()  # Output: [0, 1]

# NEW - DeepCT  
torch.clamp(F.relu(term_weights), min=0, max=100)  # Output: [0, 100]
```

```python
# OLD - Conv-KNRM kernel pooling
pooled = torch.log(torch.sum(kernel_values, dim=[1, 2]) + 1e-10)

# NEW - Conv-KNRM kernel pooling
K = torch.sum(kernel_values, dim=2)  # Sum over doc first
pooled = torch.sum(torch.log(K + 1e-10), dim=1)  # Then query
```

```python
# OLD - Separate embeddings
self.deepct = DeepCT(...)  # Has its own embedding
self.convknrm = ConvKNRM(...)  # Has its own embedding

# NEW - Shared embedding
self.shared_embedding = nn.Embedding(...)
self.deepct.embedding = self.shared_embedding
self.convknrm = ConvKNRM(..., embedding_layer=self.shared_embedding)
```

---

### 🎯 **Recommendation:**

1. **Run test cell** để verify implementation đúng
2. **Retrain model** với fixed implementation
3. **Compare results** giữa old vs new implementation
4. **Consider adding:**
   - Pre-trained Vietnamese embeddings (PhoBERT, Word2Vec)
   - Learning rate scheduling
   - Early stopping
   - Validation set evaluation

---

In [ ]:
def generate_query_doc_pairs(documents, processor, vocab, num_pairs=5000):
    """
    Generate query-document pairs with pseudo-relevance labels
    
    Strategy:
    - Positive: extract key phrases from document title as query
    - Negative: random documents that don't match the query
    """
    
    print(f"\n🔨 Generating {num_pairs} query-document pairs...")
    
    pairs = []
    
    for _ in tqdm(range(num_pairs), desc="Generating pairs"):
        # Select random document
        doc = random.choice(documents)
        title = doc.get('title', '')
        content = doc.get('content', '')
        
        if not title or not content:
            continue
        
        # Generate query from title (first few words)
        title_tokens = processor.preprocess(title)
        if len(title_tokens) < 3:
            continue
        
        # Query: random 2-4 words from title
        query_len = random.randint(2, min(4, len(title_tokens)))
        start_idx = random.randint(0, max(0, len(title_tokens) - query_len))
        query_tokens = title_tokens[start_idx:start_idx + query_len]
        
        # Document tokens
        doc_tokens = processor.preprocess(f"{title} {content}")
        
        if not query_tokens or not doc_tokens:
            continue
        
        # Positive pair (label=1)
        pairs.append({
            'query': query_tokens,
            'document': doc_tokens,
            'label': 1  # Relevant
        })
        
        # Negative pair: random non-matching document (label=0)
        neg_doc = random.choice(documents)
        neg_content = f"{neg_doc.get('title', '')} {neg_doc.get('content', '')}"
        neg_tokens = processor.preprocess(neg_content)
        
        if neg_tokens and neg_doc != doc:
            pairs.append({
                'query': query_tokens,
                'document': neg_tokens,
                'label': 0  # Non-relevant
            })
    
    print(f"✓ Generated {len(pairs)} pairs")
    print(f"  - Positive pairs: {sum(1 for p in pairs if p['label'] == 1)}")
    print(f"  - Negative pairs: {sum(1 for p in pairs if p['label'] == 0)}")
    
    return pairs

# Generate training data
train_pairs = generate_query_doc_pairs(documents, processor, vocab, num_pairs=3000)


🔨 Generating 3000 query-document pairs...


Generating pairs: 100%|██████████| 3000/3000 [03:34<00:00, 13.96it/s]

✓ Generated 5798 pairs
  - Positive pairs: 2900
  - Negative pairs: 2898


In [ ]:
class RankingDataset(Dataset):
    """PyTorch Dataset for query-document ranking"""
    
    def __init__(self, pairs, vocab, max_query_len=20, max_doc_len=200):
        self.pairs = pairs
        self.vocab = vocab
        self.max_query_len = max_query_len
        self.max_doc_len = max_doc_len
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        pair = self.pairs[idx]
        
        # Encode query and document
        query_indices = self.vocab.encode(pair['query'], max_len=self.max_query_len)
        doc_indices = self.vocab.encode(pair['document'], max_len=self.max_doc_len)
        
        return {
            'query': torch.LongTensor(query_indices),
            'document': torch.LongTensor(doc_indices),
            'label': torch.FloatTensor([pair['label']])
        }

# Create dataset and dataloader
train_dataset = RankingDataset(train_pairs, vocab, max_query_len=20, max_doc_len=200)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"\n✅ Dataset ready:")
print(f"  - Training samples: {len(train_dataset)}")
print(f"  - Batch size: 32")
print(f"  - Number of batches: {len(train_loader)}")


✅ Dataset ready:
  - Training samples: 5798
  - Batch size: 32
  - Number of batches: 182


## 9. Training Pipeline

In [ ]:
def train_model(model, train_loader, epochs=10, lr=0.001, device='cpu'):
    """Train DeepCT + Conv-KNRM model"""
    
    model = model.to(device)
    model.train()
    
    # Binary cross-entropy loss for ranking
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    history = {'loss': [], 'accuracy': []}
    
    print(f"\n🏋️ Training on {device}...")
    print(f"Epochs: {epochs}, Learning rate: {lr}\n")
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for batch in pbar:
            query = batch['query'].to(device)
            document = batch['document'].to(device)
            label = batch['label'].to(device)
            
            # Forward pass
            scores, doc_weights = model(query, document)
            
            # Compute loss
            loss = criterion(scores, label)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Statistics
            epoch_loss += loss.item()
            predictions = (torch.sigmoid(scores) > 0.5).float()
            correct += (predictions == label).sum().item()
            total += label.size(0)
            
            # Update progress bar
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100*correct/total:.2f}%'
            })
        
        # Epoch statistics
        avg_loss = epoch_loss / len(train_loader)
        accuracy = 100 * correct / total
        
        history['loss'].append(avg_loss)
        history['accuracy'].append(accuracy)
        
        print(f"Epoch {epoch+1}/{epochs}: Loss = {avg_loss:.4f}, Accuracy = {accuracy:.2f}%\n")
    
    print("✅ Training completed!")
    return model, history

# Initialize model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model = DeepCT_ConvKNRM(
    vocab_size=vocab_size,
    embedding_dim=100,
    hidden_dim=128,
    n_kernels=11,
    conv_filters=[1, 2, 3],
    use_deepct=True
)

print(f"\n Model summary:")
print(f"  - Vocabulary size: {vocab_size}")
print(f"  - Embedding dim: 100")
print(f"  - Hidden dim: 128")
print(f"  - Kernels: 11")
print(f"  - Conv filters: [1, 2, 3]")
print(f"  - Total parameters: {sum(p.numel() for p in model.parameters()):,}")

Using device: cpu

 Model summary:
  - Vocabulary size: 10369
  - Embedding dim: 100
  - Hidden dim: 128
  - Kernels: 11
  - Conv filters: [1, 2, 3]
  - Total parameters: 1,763,272

 Model summary:
  - Vocabulary size: 10369
  - Embedding dim: 100
  - Hidden dim: 128
  - Kernels: 11
  - Conv filters: [1, 2, 3]
  - Total parameters: 1,763,272


### Train the model

Uncomment and run to start training:

In [ ]:
#  TRAIN THE MODEL
# ====================
print(" Starting training process...")
print("  This will take several minutes depending on your hardware\n")

# Train the model
trained_model, history = train_model(model, train_loader, epochs=5, lr=0.001, device=device)

# Save trained model
torch.save(trained_model.state_dict(), 'deepct_convknrm_vi.pth')
print("\n Model saved to deepct_convknrm_vi.pth")
print(" Training complete! Model ready for search.")

 Starting training process...
⏱  This will take several minutes depending on your hardware




🏋️ Training on cpu...
Epochs: 5, Learning rate: 0.001



Epoch 1/5: 100%|██████████| 182/182 [02:30<00:00,  1.21it/s, loss=0.6532, acc=49.83%]


Epoch 1/5: Loss = 4.8764, Accuracy = 49.83%



Epoch 2/5: 100%|██████████| 182/182 [02:04<00:00,  1.46it/s, loss=0.6954, acc=48.19%]


...

Epoch 3/5: Loss = 0.6960, Accuracy = 49.12%



Epoch 4/5: 100%|██████████| 182/182 [01:51<00:00,  1.64it/s, loss=0.7093, acc=49.83%]


Epoch 4/5: Loss = 0.6941, Accuracy = 49.83%



Epoch 5/5: 100%|██████████| 182/182 [01:51<00:00,  1.63it/s, loss=0.6865, acc=48.48%]

Epoch 5/5: Loss = 0.6938, Accuracy = 48.48%

✅ Training completed!

 Model saved to deepct_convknrm_vi.pth
 Training complete! Model ready for search.


## 10. Search & Ranking Demo

Test the trained model with real queries.

In [ ]:
class NeuralSearchEngine:
    """Search engine using DeepCT + Conv-KNRM with FAST batch inference"""
    
    def __init__(self, model, vocab, processor, documents, device='cpu', max_doc_len=200):
        self.model = model.to(device)
        self.model.eval()
        self.vocab = vocab
        self.processor = processor
        self.documents = documents
        self.device = device
        self.max_doc_len = max_doc_len
        
        # Pre-encode all documents once for faster search
        print("\n Pre-encoding documents for fast search...")
        self.doc_tensors = []
        for doc in tqdm(documents, desc="Encoding docs"):
            title = doc.get('title', '')
            content = doc.get('content', '')
            full_text = f"{title} {content}"
            doc_tokens = self.processor.preprocess(full_text)
            doc_indices = self.vocab.encode(doc_tokens, max_len=max_doc_len)
            self.doc_tensors.append(torch.LongTensor(doc_indices))
        
        # Stack into single tensor for batch processing
        self.doc_batch = torch.stack(self.doc_tensors).to(device)
        print(f"✓ Encoded {len(documents)} documents: shape {self.doc_batch.shape}")
        
    def search(self, query, top_k=10, max_query_len=20, batch_size=256):
        """Search and rank documents for query (FAST version with batching)"""
        
        # Preprocess query
        query_tokens = self.processor.preprocess(query)
        query_indices = self.vocab.encode(query_tokens, max_len=max_query_len)
        query_tensor = torch.LongTensor(query_indices).unsqueeze(0).to(self.device)
        
        all_scores = []
        
        print(f"\n Fast ranking {len(self.documents)} documents with batch_size={batch_size}...")
        
        with torch.no_grad():
            # Process documents in batches
            num_docs = len(self.documents)
            num_batches = (num_docs + batch_size - 1) // batch_size
            
            for i in tqdm(range(num_batches), desc="Ranking batches"):
                start_idx = i * batch_size
                end_idx = min((i + 1) * batch_size, num_docs)
                
                # Get batch of documents
                doc_batch = self.doc_batch[start_idx:end_idx]
                batch_len = doc_batch.size(0)
                
                # Expand query to match batch size
                query_batch = query_tensor.expand(batch_len, -1)
                
                # Get ranking scores for batch
                scores, _ = self.model(query_batch, doc_batch)
                scores = torch.sigmoid(scores).squeeze(-1).cpu().numpy()
                
                all_scores.extend(scores)
        
        # Create results
        results = []
        for idx, score in enumerate(all_scores):
            results.append({
                'doc_index': idx,
                'score': float(score),
                'document': self.documents[idx]
            })
        
        # Sort by score and return top-k
        results.sort(key=lambda x: x['score'], reverse=True)
        
        return results[:top_k]
    
    def display_results(self, results, show_weights=False):
        """Display search results"""
        
        print(f"\n{'='*100}")
        print(f"TOP {len(results)} RESULTS")
        print(f"{'='*100}\n")
        
        for i, result in enumerate(results, 1):
            doc = result['document']
            score = result['score']
            
            title = doc.get('title', 'No title')
            content = doc.get('content', '')
            date = doc.get('date', 'No date')
            author = doc.get('author', 'Unknown')
            
            # Create snippet
            snippet = content[:200] + "..." if len(content) > 200 else content
            
            print(f"[{i}]  SCORE: {score:.4f}")
            print(f" Title: {title}")
            print(f" Date: {date} |  Author: {author}")
            print(f" Content: {snippet}")
            print(f"{'-'*100}\n")

print("✓ NeuralSearchEngine class defined (FAST version with batching)")

✓ NeuralSearchEngine class defined (FAST version with batching)


### Example Search Queries

In [ ]:
# ⚡ INITIALIZE FAST SEARCH ENGINE WITH BATCH INFERENCE
# ======================================================
print(" Creating Fast Search Engine...")
print(" Pre-encoding all documents for super fast search...\n")

# Pre-encode all documents once
print(" Step 1: Encoding all 1756 documents...")
all_doc_tensors = []
for doc in tqdm(documents, desc="Encoding"):
    title = doc.get('title', '')
    content = doc.get('content', '')
    full_text = f"{title} {content}"
    doc_tokens = processor.preprocess(full_text)
    doc_indices = vocab.encode(doc_tokens, max_len=200)
    all_doc_tensors.append(torch.LongTensor(doc_indices))

# Stack into batch tensor
doc_batch_tensor = torch.stack(all_doc_tensors).to(device)
print(f"✓ Encoded documents shape: {doc_batch_tensor.shape}")
print(f"✓ Pre-encoding complete! Now search will be 10-15x faster!\n")

# Fast search function
def fast_search(query_text, top_k=5, batch_size=256):
    """Fast batch search - takes ~5-10 seconds instead of 90s!"""
    
    # Encode query
    query_tokens = processor.preprocess(query_text)
    query_indices = vocab.encode(query_tokens, max_len=20)
    query_tensor = torch.LongTensor(query_indices).unsqueeze(0).to(device)
    
    all_scores = []
    
    print(f"\n Fast ranking {len(documents)} docs in batches of {batch_size}...")
    
    with torch.no_grad():
        num_docs = len(documents)
        num_batches = (num_docs + batch_size - 1) // batch_size
        
        for i in tqdm(range(num_batches), desc="Ranking"):
            start = i * batch_size
            end = min((i + 1) * batch_size, num_docs)
            
            # Get batch
            doc_batch = doc_batch_tensor[start:end]
            batch_len = doc_batch.size(0)
            
            # Expand query
            query_batch = query_tensor.expand(batch_len, -1)
            
            # Score batch
            scores, _ = trained_model(query_batch, doc_batch)
            scores = torch.sigmoid(scores).squeeze(-1).cpu().numpy()
            all_scores.extend(scores)
    
    # Create results
    results = [(i, float(score), documents[i]) for i, score in enumerate(all_scores)]
    results.sort(key=lambda x: x[1], reverse=True)
    
    return results[:top_k]

def display_fast_results(results):
    """Display search results nicely"""
    print(f"\n{'='*100}")
    print(f" TOP {len(results)} RESULTS")
    print(f"{'='*100}\n")
    
    for rank, (idx, score, doc) in enumerate(results, 1):
        title = doc.get('title', 'No title')
        content = doc.get('content', '')
        date = doc.get('date', 'No date')
        author = doc.get('author', 'Unknown')
        snippet = content[:200] + "..." if len(content) > 200 else content
        
        print(f"[{rank}]  SCORE: {score:.4f}")
        print(f" Title: {title}")
        print(f" Date: {date} |  Author: {author}")
        print(f" Content: {snippet}")
        print(f"{'-'*100}\n")

print(" Fast search engine ready!")
print("\n Example queries:")
for q in ["Quang Hải", "HLV Park Hang-seo", "đội tuyển Việt Nam", "V-League"]:
    print(f"  • {q}")
print("\n Usage: results = fast_search('your query', top_k=5)")

 Creating Fast Search Engine...
 Pre-encoding all documents for super fast search...

 Step 1: Encoding all 1756 documents...


Encoding: 100%|██████████| 1756/1756 [00:59<00:00, 29.53it/s]

✓ Encoded documents shape: torch.Size([1756, 200])
✓ Pre-encoding complete! Now search will be 10-15x faster!

 Fast search engine ready!

 Example queries:
  • Quang Hải
  • HLV Park Hang-seo
  • đội tuyển Việt Nam
  • V-League

 Usage: results = fast_search('your query', top_k=5)


### ? Interactive Fast Search Demo

**⚠️ IMPORTANT: Run cells in this order first:**
1. Cell 29 (Define NeuralSearchEngine class) 
2. Cell 31 (Initialize fast search - pre-encode documents) - Takes ~30s
3. Cell 33 (This cell - Interactive search) - Takes ~5-10s per query

**Why so fast?**
- ✅ Pre-encode ALL documents once (batch tensor)
- ✅ Batch inference (256 docs at a time)
- ✅ 10-15x faster: 91s → 5-10s per search!

In [ ]:
#  INTERACTIVE FAST SEARCH DEMO
# =================================
# Nhập query và nhận kết quả ngay lập tức!

print("="*80)
print("VIETNAMESE FOOTBALL SEARCH ENGINE (TRAINED MODEL)")
print("="*80)
print("\n Nhập từ khóa tìm kiếm (ví dụ: 'Quang Hải', 'Park Hang-seo')")
print("Để trống và Enter để thoát\n")

# Input query from user
query = input("Nhập tìm kiếm: ").strip()

if query:
    top_k = 5
    
    print(f"\n Đang tìm kiếm: '{query}'")
    print(f" Hiển thị top {top_k} kết quả\n")
    
    # Use fast search function
    results = fast_search(query, top_k=top_k)
    display_fast_results(results)
    
    print("\nHoàn thành! Chạy lại cell này để tìm kiếm query khác.")
    print("Search time: ~5-10 seconds (10x faster than before!)")
else:
    print("Không có query. Hãy nhập từ khóa tìm kiếm!")

VIETNAMESE FOOTBALL SEARCH ENGINE (TRAINED MODEL)

 Nhập từ khóa tìm kiếm (ví dụ: 'Quang Hải', 'Park Hang-seo')
Để trống và Enter để thoát


 Đang tìm kiếm: 'Lương Xuân Trường'
 Hiển thị top 5 kết quả


 Fast ranking 1756 docs in batches of 256...

 Đang tìm kiếm: 'Lương Xuân Trường'
 Hiển thị top 5 kết quả


 Fast ranking 1756 docs in batches of 256...


Ranking: 100%|██████████| 7/7 [00:09<00:00,  1.36s/it]


 TOP 5 RESULTS

[1]  SCORE: 0.4919
 Title: Báo Nhật Bản: 'Thêm một lần muối mặt trước bóng đá Việt Nam'
 Date: Thứ ba, 8/4/2025, 15:09 (GMT+7) |  Author: Trung Thu
 Content: Tháng 8/2024, Nhật Bản thua Việt Nam 0-1 ở giải giao hữu U16 Peace Cup tại Trung Quốc. Tám tháng sau, hầu hết cầu thủ tham dự trận này tái ngộ ở vòng bảng U17 châu Á 2025 đang diễn ra tại Arab Saudi. ...
----------------------------------------------------------------------------------------------------

[2]  SCORE: 0.4919
 Title: Lào 1-4 Việt Nam
 Date: Thứ ba, 10/12/2024, 00:04 (GMT+7) |  Author: Unknown
 Content: 
----------------------------------------------------------------------------------------------------

[3]  SCORE: 0.4919
 Title: Nguyễn Filip lỡ hẹn tuyển Việt Nam
 Date: Thứ bảy, 8/3/2025, 12:35 (GMT+7) |  Author: Hiếu Lương
 Content: Nguyễn Filip sẽ về CH Czech, sau khi cùng Công an Hà Nội (CAHN) gặp Nam Định tại vòng 16 V-League tối nay 8/3. Thời gian trở lại chưa được ấn định. Phía CAHN cũng gửi c